In [4]:
import pandas as pd

In [5]:
df = pd.read_excel('Data.xlsx')
df.head(5)

,incident_number,application,error_code,microservice,description,detailed_description,combined_text,resolution_comments
0,INC000005184952,NaN,NaN,NaN,NaN,AP:PRD:UR: Authorization has failed\n\nAlert T...,AP:PRD:UR: Authorization has failed\n\nAle...,Changed Group assignment to Universal Reposito...
1,INC000005183351,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,DIGITAL_PLATFORM ServiceException ur-api-gatew...,Changed Group assignment to Universal Reposito...
2,INC000005183248,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,DIGITAL_PLATFORM ServiceException ur-api-gatew...,Changed Group assignment to Universal Reposito...
3,INC000005183246,DIGITAL_PLATFORM,ServiceException,ur-api-gateway,Cassandra timeout during read query at consist...,Stage:prod\nMicroservice:\tur-api-gateway\nTra...,DIGITAL_PLATFORM ServiceException ur-api-gatew...,Changed Group assignment to Universal Reposito...
4,INC000005180860,Customer Communications->Digital->Presentment ...,NaN,NaN,NaN,Summary of the issue: ICMA - removal of digita...,Customer Communications->Digital->Presentment ...,Changed Group assignment to EDH CSM9 Assigned ...


In [9]:
df['combined_text'] = df["application"] + df["error_code"] + df["description"] + df["detailed_description"]

In [10]:
df['combined_text'].head()

0                                                  NaN
1    DIGITAL_PLATFORMServiceExceptionCassandra time...
2    DIGITAL_PLATFORMServiceExceptionCassandra time...
3    DIGITAL_PLATFORMServiceExceptionCassandra time...
4                                                  NaN
Name: combined_text, dtype: object

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Generate embeddings
embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True)

In [ ]:
import faiss
import numpy as np

# Number of dimensions for the embeddings
d = embeddings.shape[1]

# Build the index
index = faiss.IndexFlatL2(d)
index.add(embeddings.astype(np.float32))

# Example: searching for the 5 most similar items to a query
def search(query, k=5):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding.astype(np.float32), k)
    return df.iloc[I[0]]

# Testing the search function
results = search("example error code in application due to specific condition")
print(results['resolution_comments'])


In [ ]:
# Save FAISS index
faiss.write_index(index, 'faiss_index.bin')

# Load FAISS index
index = faiss.read_index('faiss_index.bin')